In [6]:
# Libraries for building network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils import data

# Libraries for dataset
import torchvision
import torchvision.transforms as transforms

# Miscellaneous Libraries
import time

# For DataLoader
from torch.utils.data import Dataset

# For Dataset Files
import os

In [7]:
# Define Hyperparameters

epochs = 2

batch_size = 32

lr = 0.01

dim = 10

momentum = 0.5

In [8]:
label_dict = []
with open("files/labels.txt", "r") as label:
    label_dict = eval(label.read())
    
file_dict = []
with open("files/partition.txt", "r") as file:
    file_dict = eval(file.read())
        


In [9]:
class Dataset(data.Dataset):
    def __init__(self, list_IDs, labels):
        self.labels = labels
        self.list_IDs = list_IDs

    def __len__(self):
        return len(self.list_IDs)

    def __getitem__(self, index):
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        # MAKE SURE THAT THE ASL DATASET IMGS ARE NAMED IN THE SAME DIRECTORY AS THIS CODE AND
        # NAMED 'ASL_Combined_Dataset/'
        X = torch.load('ASL_Combined_Dataset/' + ID)
        y = self.labels[ID]

        return X, y

In [10]:
# Modeled off https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}
max_epochs = 100

# Datasets
partition = file_dict
labels = label_dict

# Generators
training_set = Dataset(partition['train'], labels)
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(partition['validation'], labels)
validation_generator = data.DataLoader(validation_set, **params)

# MODEL IMPLEMENTATION BELOW: SEE https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

# # Loop over epochs
# for epoch in range(max_epochs):
#     # Training
#     for local_batch, local_labels in training_generator:
#         # Transfer to GPU
#         local_batch, local_labels = local_batch.to(device), local_labels.to(device)

#         # Model computations
#         [...]

#     # Validation
#     with torch.set_grad_enabled(False):
#         for local_batch, local_labels in validation_generator:
#             # Transfer to GPU
#             local_batch, local_labels = local_batch.to(device), local_labels.to(device)